# Writer Tools


This notebook provides a quick overview for getting started with Writer [tools](/docs/concepts/tools/). For detailed documentation of all Writer features and configurations head to the [Writer docs](https://dev.writer.com/home).

## Overview

### Integration details

| Class                                                                                                                                       | Package          | Local | Serializable | JS support |                                        Package downloads                                         |                                        Package latest                                         |
|:--------------------------------------------------------------------------------------------------------------------------------------------|:-----------------| :---: | :---: |:----------:|:------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------:|
| [GraphTool](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_writer.tool.GraphTool.html) | [langchain-writer](https://api.python.langchain.com/en/latest/community_api_reference.html) |      ❌       |                                       ❌                                       | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-writer?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-writer?style=flat-square&label=%20) |

### Features

We provide usage of two types of tools during invocation of ChatWriter: __function__ and __graph__.

#### Function

Most widespread type of tool. The function which allows LLM to call external APIs, fetch data from databases, and generally perform any operations you want to do. Visit our [functions docs](https://dev.writer.com/api-guides/tool-calling#tool-calling) for additional information.

#### Graph

Writer graph-based retrieval-augmented generation (RAG). By leveraging knowledge graphs, this tool enables developers to create, manage, and use structured data to enhance AI capabilities, particularly in RAG scenarios. Visit our [graph docs](https://dev.writer.com/api-guides/knowledge-graph) for additional information.

### Credentials

Head to [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) to sign and generate an API key. Once you've done this set the WRITER_API_KEY environment variable. It's necessary to create ChatWriter instance:

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

## Instantiation

Here we show how to bind Writer Graph Tool and Function tools to WriterChat:

### Graph Tool

To bind graph tool firs tof all you have to create and initialize GraphTool instance. The requirement argument - is __ID's of graphs__ you want to use as source of data:

In [ ]:
from langchain_writer.tools import GraphTool
from langchain_writer.chat_models import ChatWriter

chat = ChatWriter()

graph_id = getpass.getpass("Enter Writer Knowledge Graph ID: ")
graph_tool = GraphTool(
    graph_ids = [graph_id]
)


### Function

As usually, you can bind function tool as dict, BaseTool or BaseModel subclass or even callable method by itself:

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from typing import Optional


@tool
def get_supercopa_trophies_count(club_name: str) -> Optional[int]:
    """Returns information about supercopa trophies count.

    Args:
        club_name: Club you want to investigate info of supercopa trophies about

    Returns:
        Number of supercopa trophies or None if there is no info about requested club
    """

    if club_name == "Barcelona":
        return 15
    elif club_name == "Real Madrid":
        return 13
    elif club_name == "Atletico Madrid":
        return 2
    else:
        return None


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


get_product_info = {
    "type": "function",
    "function": {
        "name": "get_product_info",
        "description": "Get information about a product by its id",
        "parameters": {
            "type": "object",
            "properties": {
                "product_id": {
                    "type": "number",
                    "description": "The unique identifier of the product to retrieve information for",
                }
            },
            "required": ["product_id"],
        },
    },
}

Then you can simply bind all tools to chat instance:

In [ ]:
chat.bind_tools([graph_tool, get_supercopa_trophies_count, GetWeather, get_product_info])

All tools stored in __tools__ attribute of ChatWriter instance, and tool_choice mode at __tool_choice__ ('auto' by default) attribute:

In [ ]:
chat.tools

In [ ]:
chat.tool_choice

## Invocation

Model will automatically choose function while invocation (with all modes: stream/nonstream sync/async). In case of __function__ tools you wil receive assistant message with tool call request, and you have to handle it manually. In case of __graph__ tool model will call it remotely automatically and return usage info in ___additional_kwargs___ param under ___graph_data___ key:

In [ ]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
        "Use knowledge graph tool to compose this answer. Tell me what th first line of documents stored in your KG. Also I want to know: how many SuperCopa trophies have Barcelona won?"
    )
]

response = chat.invoke(messages)
messages.append(response)

In [ ]:
print(response.tool_calls)

In [ ]:
print(response.additional_kwargs["graph_data"])

In [ ]:
print(response.content)

Then you can manually handle tool call request, send to model and receive final response:

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = {
        "get_supercopa_trophies_count": get_supercopa_trophies_count,
    }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

response = chat.invoke(messages)

In [ ]:
print(response.content)

## API reference
For detailed documentation of all GraphTool features and configurations head to the [API reference](https://python.langchain.com/api_reference/writer/tools/langchain_writer.tools.GraphTool.html#langchain_writer.tools.GraphTool)